# LLaMA-Factory webui in Singularity container

## 初始環境設定
- 重新reload kernel, 並切換到 kernel Image_S-work-LlamaFactory_c00cjz00_cuda11.8_pytorch_2.1.2-cuda11.8-cudnn8-devel

In [1]:
# 初始環境設定
import os
from pathlib import Path
HOME = str(Path.home())
Add_Binarry_Path=HOME+'/.local/bin'
os.environ['PATH']=os.environ['PATH']+':'+Add_Binarry_Path

In [2]:
# GPU確認
!nvidia-smi
import torch
torch.cuda.is_available()

Tue Mar 12 07:37:26 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:1B:00.0 Off |                    0 |
| N/A   33C    P0    41W / 300W |      3MiB / 32768MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  Off  | 00000000:3D:00.0 Off |                    0 |
| N/A   

True

In [4]:
# 套件安裝 (此映像檔已經安裝以下套件, 若使用其他映像檔, 請移除 ## 安裝以下套件)
##!pip install llmtuner==0.5.3 deepspeed==0.13.1  bitsandbytes==0.42.0 -q 
##!pip install deepspeed==0.13.1 -q
##!pip install bitsandbytes==0.42.0 -q

In [5]:
## 下載 LLaMA-Factory 套件
!rm -rf LLaMA-Factory
!git clone https://github.com/hiyouga/LLaMA-Factory.git

Cloning into 'LLaMA-Factory'...
remote: Enumerating objects: 7601, done.
remote: Counting objects: 100% (425/425), done.
remote: Compressing objects: 100% (211/211), done.
remote: Total 7601 (delta 279), reused 337 (delta 212), pack-reused 7176
Receiving objects: 100% (7601/7601), 205.39 MiB | 17.76 MiB/s, done.
Resolving deltas: 100% (5498/5498), done.
Updating files: 100% (164/164), done.


In [3]:
%%bash
## 啟動 Llama-Factory Web UI
## 請記得修改最後一行, 給予huggingface token,  HF_TOKEN='hf_' ?
## 記錄檔案為 llamafactory.log 

# IP
node_ip=$(cat /etc/hosts |grep "$(hostname -a)" | awk '{print $1}')
# PORT
noed_port_genai=$(python -c "import socket; s = socket.socket(socket.AF_INET, socket.SOCK_STREAM); s.bind(('', 0)); addr = s.getsockname(); s.close(); print(addr[1])")
# PROXY
proxy_url=/rstudio/${node_ip}/${noed_port_genai}
# URL
https_url=https://node01.biobank.org.tw${proxy_url}/

# SCRIPT FILE
cat << EOF >  LLaMA-Factory/src/train_web_demo.py
from llmtuner import create_ui
import os
os.environ["WANDB_DISABLED"] = "true"

#print("${https_url}")

def main():
    demo = create_ui()
    demo.queue()
    demo.launch(server_port=${noed_port_genai}, server_name="$(hostname -s)", share=False, inbrowser=True, root_path="${proxy_url}", auth=("nchc", "nchcorgtw"))

if __name__ == "__main__":
    main()
EOF

# 參除
ps -ef |grep train_web_demo | awk '{print $2}' | xargs kill -9
sleep 1

#請更新以下 HF_TOKEN
echo $https_url
echo "Account: nchc"
echo "Password: nchcorgtw"
cd LLaMA-Factory
CUDA_VISIBLE_DEVICES=1 HF_TOKEN='hf_' nohup python src/train_web_demo.py > ../llamafactory.log 2>&1 &

kill: (169542): No such process


https://node01.biobank.org.tw/rstudio/172.16.124.152/58665/
Account: nchc
Password: nchcorgtw


kill: (170272): No such process
